In [ ]:
import os
from glob import glob
import model.aotgan 
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from tqdm import tqdm
from torchinfo import summary
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "4" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [49]:
params={'image_size':512,
        'rates':[1, 2, 4, 8],
        'block_num':8,
        'model':'aotgan',
        'gan_type':"smgan",
        'lrg':1e-4,
        'lrd':1e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':4,
        'epochs':100,
        }

In [50]:
netG =model.aotgan.InpaintGenerator(params).to(device)
optimG = torch.optim.Adam(netG.parameters(), lr=params['lrg'], betas=(params['beta1'], params['beta2']))

netD = model.aotgan.Discriminator().to(device)
optimD = torch.optim.Adam(netD.parameters(), lr=params['lrd'], betas=(params['beta1'], params['beta2']))

In [51]:
summary(netG,[(params['batch_size'],3,params['image_size'],params['image_size']),(params['batch_size'],1,params['image_size'],params['image_size'])])

Layer (type:depth-idx)                        Output Shape              Param #
InpaintGenerator                              [4, 3, 512, 512]          --
├─Sequential: 1-1                             [4, 256, 128, 128]        --
│    └─ReflectionPad2d: 2-1                   [4, 4, 518, 518]          --
│    └─Conv2d: 2-2                            [4, 64, 512, 512]         12,608
│    └─ReLU: 2-3                              [4, 64, 512, 512]         --
│    └─Conv2d: 2-4                            [4, 128, 256, 256]        131,200
│    └─ReLU: 2-5                              [4, 128, 256, 256]        --
│    └─Conv2d: 2-6                            [4, 256, 128, 128]        524,544
│    └─ReLU: 2-7                              [4, 256, 128, 128]        --
├─Sequential: 1-2                             [4, 256, 128, 128]        --
│    └─AOTBlock: 2-8                          [4, 256, 128, 128]        --
│    │    └─Sequential: 3-1                   [4, 64, 128, 128]         147,520
│

In [45]:
len(rates)

7